<a href="https://colab.research.google.com/github/PhatthanhtuHN/ML_Echocardiographic/blob/main/ML_echocardiographic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from torch import nn

from torch.utils.data import DataLoader

from torchvision import transforms
import torchvision

import numpy as np
import cv2

from torchsummary import summary

from collections import  namedtuple

import matplotlib.pyplot as plt

from google.colab import drive

from sklearn.metrics import classification_report

In [4]:
drive.mount('/content/drive')

# Đặt đường dẫn cho thư mục train và test từ dữ liệu Drive đã được kết nối 
train_dir = '/content/drive/MyDrive/Colab Notebooks/DATA_CHAMBER_2021/train'
test_dir = '/content/drive/MyDrive/Colab Notebooks/DATA_CHAMBER_2021/test'

Mounted at /content/drive


In [5]:
# Hàm get_classes() trả về ba lớp 2C, 3C, 4C tương ứng với ba loại mặt cắt của ảnh siêu âm tim
def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

TrainTest = namedtuple('TrainTest', ['train', 'test'])

# Hàm chuẩn bị dữ liệu
def prepare_data():
  img_size = 224
  # Hàm transform chuẩn hóa ảnh về kích thước 224 x 224 và chuyển thành Tensor là một mảng nhiều chiều
  transforms_train = transforms.Compose([
    transforms.Resize((img_size, img_size)),  
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225]) 
  ])
  transforms_train_augmentation = transforms.Compose([
      transforms.Resize((img_size, img_size)),
      transforms.RandomHorizontalFlip(),
      transforms.RandomVerticalFlip(),
      transforms.RandomRotation(degrees=10),
      transforms.ToTensor(), 
      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225])    
  ])
  # transforms_train_preprocess = transforms.Compose([
  #     transform.Resize((img_size, img_size)),

  # ])
  transforms_test = transforms.Compose([
    transforms.Resize((img_size, img_size)), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225]) 
  ])
  # Chuẩn bị 2 bộ dữ liệu trainset và testset với đường dẫn như trên, gán nhãn cho ảnh theo tên thư mục và trả về bộ giá trị (tuple) TrainTest gồm hai bộ dữ liệu này
  trainset = torchvision.datasets.ImageFolder(root=train_dir, transform=transforms_train)
  trainset = torchvision.datasets.ImageFolder(root=train_dir, transform=transforms_train_augmentation)
  testset = torchvision.datasets.ImageFolder(root=test_dir, transform=transforms_test)
  print('Number of images in train:', len(trainset), '\nNumber of images in test:', len(testset))
  print('Index of classes:', trainset.class_to_idx)
  print('Type:', type(trainset[0][0]), '| size: ', trainset[0][0].shape)
  return TrainTest(train=trainset, test=testset)

# Hàm đọc bộ dữ liệu, mỗi lần lấy ra 1 batch có độ dài là 32 ảnh đế xử lý song song, nối thành tensor 4 chiều là đầu vào của mạng
def prepare_loader(datasets):
  trainloader = DataLoader(dataset=datasets.train, batch_size=32, shuffle=True, num_workers=4)
  testloader = DataLoader(dataset=datasets.test, batch_size=32, shuffle=False, num_workers=4)
  print('Number of batchs in train:', len(trainloader), '\nNumber of batchs in test:', len(testloader))
  return TrainTest(train=trainloader, test=testloader)

# Dựng mô hình mạng tích chập VGG19
class VGG19(nn.Module):
  def __init__(self):
    super().__init__()
    self.features = self._make_features()
    self.classification_head = nn.Linear(in_features=25088, out_features=3)

  def forward(self, x):
    out = self.features(x)
    out = out.view(out.size(0), -1)
    out = self.classification_head(out)
    return out

  def _make_features(self):
    config = [64, 64, 'MP', 128, 128, 'MP', 256, 256, 256, 256, 'MP', 512, 512, 512, 512, 'MP', 512, 512, 512, 512, 'MP']
    layer = []
    c_in = 3
    for c in config:
      if c == 'MP':
        layer += [nn.MaxPool2d(kernel_size=2, stride=2)]
      else:
        layer += [nn.Conv2d(in_channels=c_in, out_channels=c, kernel_size=3, stride=1, padding=1), 
                  nn.BatchNorm2d(num_features=c), 
                  nn.ReLU6(inplace=True)]
        c_in = c
    return nn.Sequential(*layer)


In [6]:
def imshow(images, labels, predicted, target_names):
  img = torchvision.utils.make_grid(images)
  plt.imshow(img.permute(1, 2, 0).cpu().numpy())
  [print(target_names[c], end=' ') for c in list(labels.cpu().numpy())]
  print()
  [print(target_names[c], end=' ') for c in list(predicted.cpu().numpy())]
  print()
  # print(target_names[list(labels.cpu().numpy())])
  # print(target_names[list(predicted.cpu().numpy())])

  # def imshow(img):
  #   img = img / 2 + 0.5     # unnormalize
  #   npimg = img.numpy()
  #   plt.imshow(np.transpose(npimg, (1, 2, 0)))
  #   plt.show()

In [7]:
# Hàm train với mỗi batch trong bộ dữ liệu
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  # Cho ảnh (đã được chia thành batch trong loader) lần lượt đi qua model, với mỗi ảnh và nhãn trong bộ dữ liệu là đầu vào của mạng.
  model.train()             
  running_loss = 0.0
  reporting_steps = 30
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    # print('Type:', type(outputs), '\nOutput shape:', outputs.size())      # Type: <class 'torch.Tensor'> Output shape: torch.Size([32, 3])
    loss = loss_func(outputs, labels)

    # Cập nhật tham số trước khi sang step khác
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    # Báo cáo sau reporting_step bước
    if i % reporting_steps == reporting_steps-1:
      print(f'Epoch {epoch} step {i} ave_loss {running_loss/reporting_steps:.4f}')
      running_loss = 0.0

# Hàm test 
def test_epoch(epoch, model, loader, device):
  ytrue =[]
  ypred = []
  # Đặt model ở chế độ đánh giá (evaluate)
  with torch.no_grad():
    model.eval()
    
    # Tính toán đầu ra cho từng ảnh, với nhãn dự đoán dựa trên số to nhất trong outputs
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      
      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())
      
  return ytrue, ypred   # Trả về nhãn thực, nhãn dự đoán của ảnh


In [10]:
def main(model_name=None):
  PATH = './' + model_name + '.pth'
  classes = get_classes()
  datasets = prepare_data()

  # img, label = datasets.train[0]
  # print(classes[label], img.size)
  # print('Number of images in train', len(datasets.train), 'Number of images in test', len(datasets.test))
  # plt.imshow(img)
  loaders = prepare_loader(datasets)
  # images, labels = iter(loaders.train).next()
  # print(images.shape, labels.shape)

  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  
  if model_name == 'vgg19':
    model = VGG19()
  elif model_name == 'resnet50':
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=False)
    model.fc = torch.nn.modules.linear.Linear(in_features=2048, out_features=3, bias=True)
  elif model_name == 'densenet121':
    model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=False)
    model.classifier = torch.nn.modules.linear.Linear(in_features=1024, out_features=3, bias=True)
  else:
    pass
  # summary(model.cuda(), (3, 224, 224))
  # images, labels = iter(loaders.train).next()
  # # print(model)
  # outputs = model(images)
  # print(outputs.shape)
  # print(outputs[0])
  # _, predicted = torch.max(outputs, dim=1)
  # print(predicted)
  # imshow(images, labels, predicted, classes)
  print('Model name: ', model_name)
  model.to(device=device)
  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  for epoch in range(20):
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    ytrue, ypred = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(ytrue, ypred, target_names=classes))


In [11]:
allmodels = ['vgg19', 'resnet50', 'densenet121']
for model in allmodels:
  main(model_name=model)


Number of images in train: 6717 
Number of images in test: 1607
Index of classes: {'2C': 0, '3C': 1, '4C': 2}
Type: <class 'torch.Tensor'> | size:  torch.Size([3, 224, 224])
Number of batchs in train: 210 
Number of batchs in test: 51


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Model name:  vgg19
Epoch 0 step 29 ave_loss 22.3346
Epoch 0 step 59 ave_loss 6.8416
Epoch 0 step 89 ave_loss 2.8800
Epoch 0 step 119 ave_loss 1.5358
Epoch 0 step 149 ave_loss 1.3027
Epoch 0 step 179 ave_loss 1.1965
Epoch 0 step 209 ave_loss 1.0613
              precision    recall  f1-score   support

          2C       0.32      0.23      0.27       409
          3C       0.47      0.20      0.28       367
          4C       0.57      0.79      0.66       831

    accuracy                           0.51      1607
   macro avg       0.45      0.41      0.40      1607
weighted avg       0.48      0.51      0.47      1607

Epoch 1 step 29 ave_loss 0.9987
Epoch 1 step 59 ave_loss 0.9721
Epoch 1 step 89 ave_loss 0.9928
Epoch 1 step 119 ave_loss 0.9763
Epoch 1 step 149 ave_loss 0.9458
Epoch 1 step 179 ave_loss 0.9735
Epoch 1 step 209 ave_loss 0.9763
              precision    recall  f1-score   support

          2C       0.36      0.95      0.53       409
          3C       0.35      0.21 

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip


Model name:  resnet50
Epoch 0 step 29 ave_loss 5.8146
Epoch 0 step 59 ave_loss 1.8329
Epoch 0 step 89 ave_loss 1.3660
Epoch 0 step 119 ave_loss 1.2720
Epoch 0 step 149 ave_loss 1.2327
Epoch 0 step 179 ave_loss 1.2126
Epoch 0 step 209 ave_loss 1.1499
              precision    recall  f1-score   support

          2C       0.28      1.00      0.44       409
          3C       0.34      0.14      0.20       367
          4C       0.00      0.00      0.00       831

    accuracy                           0.29      1607
   macro avg       0.21      0.38      0.21      1607
weighted avg       0.15      0.29      0.16      1607



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1 step 29 ave_loss 1.1451
Epoch 1 step 59 ave_loss 1.2169
Epoch 1 step 89 ave_loss 1.0474
Epoch 1 step 119 ave_loss 1.0063
Epoch 1 step 149 ave_loss 1.0585
Epoch 1 step 179 ave_loss 0.9296
Epoch 1 step 209 ave_loss 0.8174
              precision    recall  f1-score   support

          2C       1.00      0.07      0.13       409
          3C       0.40      0.54      0.46       367
          4C       0.70      0.92      0.80       831

    accuracy                           0.62      1607
   macro avg       0.70      0.51      0.46      1607
weighted avg       0.71      0.62      0.55      1607

Epoch 2 step 29 ave_loss 0.9056
Epoch 2 step 59 ave_loss 0.8412
Epoch 2 step 89 ave_loss 0.8360
Epoch 2 step 119 ave_loss 0.8317
Epoch 2 step 149 ave_loss 0.6759
Epoch 2 step 179 ave_loss 0.7067
Epoch 2 step 209 ave_loss 0.6057
              precision    recall  f1-score   support

          2C       0.29      0.93      0.44       409
          3C       0.28      0.13      0.17       367


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Model name:  densenet121
Epoch 0 step 29 ave_loss 1.1035
Epoch 0 step 59 ave_loss 0.9529
Epoch 0 step 89 ave_loss 0.9250
Epoch 0 step 119 ave_loss 0.8647
Epoch 0 step 149 ave_loss 0.7527
Epoch 0 step 179 ave_loss 0.6206
Epoch 0 step 209 ave_loss 0.6116
              precision    recall  f1-score   support

          2C       0.78      0.73      0.75       409
          3C       0.39      0.96      0.56       367
          4C       0.97      0.39      0.55       831

    accuracy                           0.60      1607
   macro avg       0.71      0.69      0.62      1607
weighted avg       0.79      0.60      0.60      1607

Epoch 1 step 29 ave_loss 0.4353
Epoch 1 step 59 ave_loss 0.4417
Epoch 1 step 89 ave_loss 0.3736
Epoch 1 step 119 ave_loss 0.3866
Epoch 1 step 149 ave_loss 0.3044
Epoch 1 step 179 ave_loss 0.2759
Epoch 1 step 209 ave_loss 0.2720
              precision    recall  f1-score   support

          2C       0.66      0.77      0.71       409
          3C       0.59      

In [12]:
#VGG19 ảnh 224 raw: [56, 60, 67, 62, 65, 63, 73, 77, 77, 72]
#VGG19 ảnh 224 normalized: [56, 60, 63, 62, 65, 68, 67, 75, 73, 71]
#VGG19 ảnh 224 có data augmentation: [43, 41, 48, 59, 53, 63, 59, 77, 74, 81]

#RESNET50 ảnh 224 có data augmentation: [29, 62, 33, 72, 66, 73, 77, 80, 77, 75, 61, 82, 66, 77, 77, 76, 78, 75, 84, 74]

#DENSENET121 ảnh 224 có data augmentation: [60, 76, 64, 72, 29, 77, 65, 81, 62, 84, 65, 75, 57, 67, 74, 78, 72, 66, 74, 85]